# Create Final Dataset for Training

Calculate running statistics for each player pre match commencment (Moving stat averages, Elo/Ranking Gradients, etc.).

In [68]:
%matplotlib inline
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', None)

Import cleaned match dataset

In [69]:
clean_data = pd.read_csv("./data/0cleanDataset.csv")
clean_data["tourney_date"] = pd.to_datetime(clean_data["tourney_date"])
clean_data.sort_values(by='tourney_date', ascending=True, inplace=True)
clean_data.head(3)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p1_rank_points,p2_rank,p2_rank_points,p1_elo,p2_elo,p1_surface_elo,p2_surface_elo,winner_rank_odds,loser_rank_odds,p1_Max,p2_Max,p1_winprob,p2_winprob,p1_set1,p1_set2,p1_set3,p1_set4,p1_set5,p2_set1,p2_set2,p2_set3,p2_set4,p2_set5,tiebreak_occurred,p1_tiebreaks_won,p2_tiebreaks_won,p1_margin1,p2_margin1,p1_margin2,p2_margin2,p1_margin3,p2_margin3,p1_margin4,p2_margin4,p1_margin5,p2_margin5,p1_bpconv,p2_bpconv,p1_bp/rg,p2_bp/rg,p1_tpw,p2_tpw,p1_sw,p2_sw,RESULT,0,1,2
4161,311.0,Queen's Club,Grass,56.0,A,2002-06-10,19.0,103566,NaN,NaN,Michael Llodra,L,190.0,FRA,22.0,102434,NaN,NaN,Vincent Spadea,R,183.0,USA,27.8,7-6(5) 7-5,3,R64,92.0,3.0,8.0,77.0,54.0,38.0,9.0,12.0,1.0,4.0,4.0,5.0,81.0,56.0,40.0,15.0,12.0,5.0,7.0,103.0,363.0,73.0,517.0,1774.737122,1768.850203,1651.204600,1688.651635,73.0,103.0,1.727,2.15,0.536625,0.463375,6.0,5.0,NaN,NaN,NaN,7.0,7.0,NaN,NaN,NaN,1.0,0.0,1.0,-1.0,1.0,-2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,0.583333,0.333333,73.0,85.0,11.0,14.0,0,311.0,0311,NaN
12268,500.0,Halle,Grass,32.0,A,2002-06-10,14.0,102318,NaN,NaN,Andrei Pavel,R,183.0,ROU,28.3,103285,NaN,Q,Radek Stepanek,R,185.0,CZE,23.5,7-6(5) 6-4,3,R32,92.0,6.0,2.0,85.0,54.0,39.0,16.0,11.0,6.0,7.0,4.0,3.0,88.0,57.0,39.0,13.0,11.0,7.0,9.0,21.0,1445.0,150.0,255.0,2004.588637,1806.463144,1713.354605,1500.000000,21.0,150.0,1.533,2.55,0.607728,0.392272,7.0,6.0,NaN,NaN,NaN,6.0,4.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0,-1.0,2.0,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,0.818182,0.636364,91.0,82.0,13.0,10.0,1,500.0,0500,NaN
12261,500.0,Halle,Grass,32.0,A,2002-06-10,12.0,101948,4.0,NaN,Pete Sampras,R,185.0,USA,30.8,102911,NaN,NaN,Andrei Stoliarov,R,178.0,RUS,25.4,4-6 6-3 6-4,3,R32,99.0,22.0,5.0,81.0,53.0,47.0,14.0,15.0,5.0,6.0,7.0,4.0,86.0,30.0,23.0,33.0,14.0,6.0,8.0,13.0,1910.0,107.0,358.0,2062.432193,1816.242324,2105.945746,1703.775361,13.0,107.0,1.143,5.50,0.813935,0.186065,4.0,6.0,6.0,NaN,NaN,6.0,3.0,4.0,NaN,NaN,0.0,NaN,NaN,-2.0,2.0,3.0,-3.0,2.0,-2.0,NaN,NaN,NaN,NaN,2.0,1.0,0.571429,0.400000,91.0,76.0,16.0,13.0,1,500.0,0500,NaN


In [71]:
clean_data[(clean_data['tourney_name'] == 'Chennai') & 
           (clean_data['round'] == 'R16') & 
           (clean_data['tourney_date'].dt.year == 2012)]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p1_rank_points,p2_rank,p2_rank_points,p1_elo,p2_elo,p1_surface_elo,p2_surface_elo,winner_rank_odds,loser_rank_odds,p1_Max,p2_Max,p1_winprob,p2_winprob,p1_set1,p1_set2,p1_set3,p1_set4,p1_set5,p2_set1,p2_set2,p2_set3,p2_set4,p2_set5,tiebreak_occurred,p1_tiebreaks_won,p2_tiebreaks_won,p1_margin1,p2_margin1,p1_margin2,p2_margin2,p1_margin3,p2_margin3,p1_margin4,p2_margin4,p1_margin5,p2_margin5,p1_bpconv,p2_bpconv,p1_bp/rg,p2_bp/rg,p1_tpw,p2_tpw,p1_sw,p2_sw,RESULT,0,1,2
34605,891.0,Chennai,Hard,28.0,A,2012-01-02,13.0,104386,1.0,NaN,Janko Tipsarevic,R,180.0,SRB,27.5,106026,NaN,WC,Yuki Bhambri,R,185.0,IND,19.4,6-1 6-3,3,R16,62.0,5.0,1.0,41.0,20.0,17.0,16.0,8.0,1.0,1.0,3.0,2.0,39.0,22.0,12.0,9.0,8.0,0.0,4.0,9.0,2595.0,345.0,122.0,2083.223763,1676.164827,2022.868289,1649.940473,9.0,345.0,1.08,13.00,0.925267,0.074733,6.0,6.0,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,0.0,NaN,NaN,5.0,-5.0,3.0,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.500000,0.125000,51.0,29.0,12.0,4.0,1,891.0,0891,NaN
22814,891.0,Chennai,Hard,28.0,A,2012-01-02,15.0,104527,3.0,NaN,Stan Wawrinka,R,183.0,SUI,26.7,104273,NaN,LL,Edouard Roger-Vasselin,R,185.0,FRA,28.0,3-6 6-3 7-5,3,R16,145.0,10.0,1.0,82.0,52.0,35.0,19.0,15.0,2.0,6.0,4.0,0.0,104.0,70.0,43.0,16.0,15.0,6.0,11.0,17.0,1820.0,106.0,548.0,2048.529412,1731.628276,2024.969550,1665.748861,17.0,106.0,1.23,4.70,0.792580,0.207420,3.0,6.0,7.0,NaN,NaN,6.0,3.0,5.0,NaN,NaN,0.0,NaN,NaN,-3.0,3.0,3.0,-3.0,2.0,-2.0,NaN,NaN,NaN,NaN,5.0,4.0,0.733333,0.400000,99.0,87.0,16.0,14.0,1,891.0,0891,NaN
34607,891.0,Chennai,Hard,28.0,A,2012-01-02,16.0,104424,NaN,Q,Go Soeda,R,178.0,JPN,27.3,104471,5.0,NaN,Ivan Dodig,R,183.0,CRO,26.9,7-6(6) 6-3,3,R16,108.0,4.0,3.0,73.0,43.0,32.0,18.0,11.0,3.0,4.0,12.0,2.0,69.0,41.0,31.0,11.0,10.0,5.0,7.0,120.0,481.0,36.0,1065.0,1790.245765,1889.089275,1738.018113,1856.661461,120.0,36.0,3.00,1.50,0.348837,0.651163,7.0,6.0,NaN,NaN,NaN,6.0,3.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0,-1.0,3.0,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,0.700000,0.363636,77.0,65.0,13.0,9.0,1,891.0,0891,NaN
34608,891.0,Chennai,Hard,28.0,A,2012-01-02,17.0,105332,NaN,NaN,Benoit Paire,R,196.0,FRA,22.6,104534,NaN,NaN,Dudi Sela,R,175.0,ISR,26.7,4-6 6-2 6-4,3,R16,121.0,10.0,5.0,104.0,58.0,40.0,21.0,14.0,4.0,7.0,3.0,2.0,87.0,60.0,44.0,17.0,14.0,6.0,7.0,95.0,586.0,83.0,640.0,1740.608912,1801.102938,1749.522191,1744.346967,83.0,95.0,2.50,1.61,0.393643,0.606357,6.0,2.0,4.0,NaN,NaN,4.0,6.0,6.0,NaN,NaN,0.0,NaN,NaN,2.0,-2.0,-4.0,4.0,-2.0,2.0,NaN,NaN,NaN,NaN,1.0,3.0,0.500000,0.500000,87.0,104.0,12.0,16.0,0,891.0,0891,NaN
12202,891.0,Chennai,Hard,28.0,A,2012-01-02,20.0,104597,2.0,NaN,Nicolas Almagro,R,183.0,ESP,26.3,104327,NaN,NaN,Steve Darcis,R,178.0,BEL,27.8,W/O,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2380.0,88.0,622.0,2023.251781,1894.393101,1870.693221,1825.186044,10.0,88.0,1.48,2.86,0.658986,0.341014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,891.0,0891,NaN
34606,891.0,Chennai,Hard,28.0,A,2012-01-02,14.0,104676,NaN,NaN,Andreas Beck,L,190.0,GER,25.9,105676,NaN,WC,David Goffin,R,180.0,BEL,21.0,4-6 6-4 6-2,3,R16,138.0,4.0,0.0,98.0,71.0,43.0,14.0,14.0,12.0,16.0,6.0,4.0,82.0,43.0,30.0,25.0,14.0,6.0,8.0,98.0,580.0,174.0,287.0,1791.006479,1719.635709,1743.242321,1700.605268,174.0,98.0,1.75,2.22,0.560606,0.439394,6.0,4.0,2.0,NaN,NaN,4.0,6.0,6.0,NaN,NaN,0.0,NaN,NaN,2.0,-2.0,-2.0,2.0,-4.0,4.0,NaN,NaN,NaN,NaN,2.0,4.0,0.571429,1.142857,84.0,96.0,12.0,16.0,0,891.0,0891,NaN
28683,891.0,Chennai,Hard,28.0,A,2012-01-02,18.0,105683,4.0,NaN,Milos Raonic,R,196.0,CAN,21.0,103812,

### Utilise updateStats file to process stats as required

In [7]:
from utils.updateStats import getStats, updateStats, createStats
final_dataset = []
prev_stats = createStats()

# Input relevant data for each match
for index, row in tqdm(clean_data.iterrows(), total=len(clean_data)):
    player1 = {
        "ID": row["p1_id"],
        "ATP_POINTS": row["p1_rank_points"],
        "ATP_RANK": row["p1_rank"],
        "AGE": row["p1_age"],
        "HEIGHT": row["p1_ht"]
    }

    player2 = {
        "ID": row["p2_id"],
        "ATP_POINTS": row["p2_rank_points"],
        "ATP_RANK": row["p2_rank"],
        "AGE": row["p2_age"],
        "HEIGHT": row["p2_ht"]
    }

    match = {
        "TOURNAMENT_LEVEL": row["tourney_level"],
        "TOURNEY_NAME": row["tourney_name"],
        "BEST_OF": row["best_of"],
        "ROUND": row["round"],
        "DRAW_SIZE": row["draw_size"],
        "SURFACE": row["surface"],
        "TOURNEY_DATE": row["tourney_date"]
    }
    
    # Retrieve player stats pre match
    output = getStats(player1, player2, match, prev_stats)
    
    # Add columns not requiring processing
    output["P1_WINPROB"] = row["p1_winprob"]
    output["P2_WINPROB"] = row["p2_winprob"]
    output["P1_ODDS"] = row["p1_Max"]
    output["P2_ODDS"] = row["p2_Max"]
    output["P1_ELO"] = row["p1_elo"]
    output["P2_ELO"] = row["p2_elo"]
    output["P1_SURFACE_ELO"] = row["p1_surface_elo"]
    output["P2_SURFACE_ELO"] = row["p2_surface_elo"]
    output["tourney_date"] = row["tourney_date"]
    output["tourney_id"] = row["tourney_id"]
    output["ROUND"] = row["round"]
    

    # Append stats to final dataset
    match_data = dict(sorted(output.items()))
    match_data["RESULT"] = row.RESULT
    final_dataset.append(match_data)

    # Update Player Stats post match
    prev_stats = updateStats(row, prev_stats)


final_dataset_odds = pd.DataFrame(final_dataset)

100%|██████████| 43278/43278 [02:49<00:00, 255.66it/s]


Calculate remaining required stats

In [8]:
final_dataset_odds['ELO_DIFF'] = final_dataset_odds['P1_ELO'] - final_dataset_odds['P2_ELO']
final_dataset_odds['SURFACE_ELO_DIFF'] = final_dataset_odds['P1_SURFACE_ELO'] - final_dataset_odds['P2_SURFACE_ELO']
final_dataset_odds["PRED_ODDS"] = (final_dataset_odds["P1_WINPROB"] > final_dataset_odds["P2_WINPROB"]).astype(int)

Export Final Dataset

In [9]:
final_dataset_odds.to_csv("./data/1finalDatasetOdds.csv", index=False)